In [1]:
!pip install --upgrade datasets fsspec huggingface_hub

from IPython import get_ipython
from IPython.display import display
from datasets import load_dataset

dataset = load_dataset("Abirate/english_quotes", split="train")
dataset = dataset.filter(lambda x: x["quote"] and x["author"])
dataset = dataset.shuffle(seed=42)
dataset = dataset.select(range(1000))

dataset[0]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.3/509.3 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 87.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.31.2
    Uninstalling huggingface-hub-0.31.2:
      Successfully uninstalled huggingface-hub-0.31.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2508 [00:00<?, ? examples/s]

{'quote': "“I don't mind making jokes, but I don't want to look like one.”",
 'author': 'Marilyn Monroe',
 'tags': ['appearance', 'jokes', 'marilyn-monroe']}

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
quotes_list = dataset["quote"]
quote_embeddings = model.encode(quotes_list, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [4]:
!pip install faiss-cpu
import faiss
import numpy as np

embedding_dim = quote_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(quote_embeddings))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 47.6 MB/s eta 0:00:00


In [5]:
def retrieve_quotes(query, dataset, top_k=5):
        query_embedding = model.encode([query])
        scores, indices = index.search(np.array(query_embedding), top_k)

        results = []

        quotes = dataset["quote"]
        authors = dataset["author"]

        for i in indices[0]:
            results.append({
                "quote": quotes[i],
                "author": authors[i],

            })
        return results

# sample:
retrieve_quotes("funny quotes about life", dataset)

[{'quote': '“love the life you live.live the life you love.”',
  'author': 'Bob Marley'},
 {'quote': '“When I was 5 years old, my mother always told me that happiness was the key to life. When I went to school, they asked me what I wanted to be when I grew up. I wrote down â€˜happyâ€™. They told me I didnâ€™t understand the assignment, and I told them they didnâ€™t understand life.”',
  'author': 'John Lennon'},
 {'quote': '“People say that life is the thing, but I prefer reading.”',
  'author': 'Logan Pearsall Smith'},
 {'quote': '“If my life is going to mean anything, I have to live it myself.”',
  'author': 'Rick Riordan,'},
 {'quote': '“The purpose of life is not to be happy. It is to be useful, to be honorable, to be compassionate, to have it make some difference that you have lived and lived well.”',
  'author': 'Ralph Waldo Emerson'}]

In [14]:
def format_as_rag_response(query, top_k=5):
    results = retrieve_quotes(query, top_k)
    response = f"🔍 **Query:** {query}\n\n"
    for r in results:
        response += f"> *{r['quote']}*  — **{r['author']}**  \n"
    return response

In [16]:
!pip install streamlit
import streamlit as st

st.title("Semantic Quote Search (RAG Style)")
query = st.text_input("Enter a query:", "")

if query:
    results = retrieve_quotes(query, top_k=5)
    for r in results:
        st.markdown(f"> *{r['quote']}*  \n\n— **{r['author']}**")
        st.markdown("---")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


2025-05-25 20:37:27.223 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 20:37:27.389 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-25 20:37:27.390 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 20:37:27.393 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 20:37:27.394 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 20:37:27.395 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 20:37:27.396 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 20:37:27.397 Session state does not 

In [3]:
%%writefile app.py

Overwriting app.py


In [4]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.196.41.145:8501

  Stopping...
  Stopping...
